In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

Hypothesis Testing
As part of your analysis, you'll conduct hypothesis testing to make data-driven conclusions about the effectiveness of the redesign. See the full details below:

1) COMPLETION RATE
2) COMPLETION RATE WITH A COST-EFFECTIVENESS THRESHOLD
3) OTHER HYPOTHESIS EXAMPLES

In [11]:
# Load your CSV or paste raw data
df = pd.read_csv("/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_cleaned_merged_web_data_pt.txt", parse_dates=["date_time"])
variation_df = pd.read_csv("/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_cleaned_experiment_clients_data.txt")
demographics_df = pd.read_csv("/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_final_demo_cleaned.txt")

1) COMPLETION RATE

In [7]:
# Step 0: Merge variation info into df by client_id
df = df.merge(variation_df, on="client_id", how="left")

# Sort actions chronologically per visit
df = df.sort_values(by=["visit_id", "date_time"])

# Remove duplicate steps per visit (to avoid loops/skips being counted multiple times)
df_unique_steps = df.drop_duplicates(subset=["visit_id", "process_step"])

# Define the correct step order for the funnel
step_order = ["start", "step_1", "step_2", "step_3", "confirm"]

# Calculate funnel conversions per Variation group
funnel_per_variation = (
    df_unique_steps.groupby(["Variation", "process_step"])["visit_id"]
    .nunique()
    .reindex(step_order, level=1, fill_value=0)  # reindex steps per variation
    .reset_index()
)

# Calculate funnel metrics per group
def compute_funnel_metrics(df_group):
    df_group = df_group.set_index("process_step").reindex(step_order, fill_value=0)
    df_group = df_group.rename(columns={"visit_id": "num_visits"})
    df_group["conversion_rate"] = df_group["num_visits"] / df_group["num_visits"].iloc[0] * 100  # from start
    df_group["step_conversion"] = df_group["num_visits"] / df_group["num_visits"].shift(1) * 100
    return df_group

# Apply for each variation
result = funnel_per_variation.groupby("Variation").apply(compute_funnel_metrics)

# Display results
for variation in result.index.levels[0]:
    print(f"\nFunnel Conversion for Variation: {variation}\n")
    print(result.loc[variation])


Funnel Conversion for Variation: Control

             Variation  num_visits  conversion_rate  step_conversion
process_step                                                        
start          Control       30741       100.000000              NaN
step_1         Control       23394        76.100322        76.100322
step_2         Control       19997        65.049933        85.479183
step_3         Control       18165        59.090466        90.838626
confirm        Control       15912        51.761491        87.597027

Funnel Conversion for Variation: Test

             Variation  num_visits  conversion_rate  step_conversion
process_step                                                        
start             Test       33017       100.000000              NaN
step_1            Test       28148        85.253051        85.253051
step_2            Test       24378        73.834691        86.606508
step_3            Test       22069        66.841324        90.528345
confirm           Te

/var/folders/_2/ksgd3nl52j9g0sxwpjjd1bgh0000gn/T/ipykernel_8200/4200738623.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = funnel_per_variation.groupby("Variation").apply(compute_funnel_metrics)


In [9]:
pivot = funnel_per_variation.pivot(index='process_step', columns='Variation', values='visit_id').fillna(0)

# Total visits per group (at start step)
total_control = pivot.loc['start', 'Control']
total_test = pivot.loc['start', 'Test']

print(f"Total Control visits: {total_control}")
print(f"Total Test visits: {total_test}\n")

print("Step-wise significance test (two-proportion z-test):\n")

for step in step_order:
    control_success = pivot.loc[step, 'Control']
    test_success = pivot.loc[step, 'Test']

    successes = np.array([test_success, control_success])
    trials = np.array([total_test, total_control])

    stat, pval = proportions_ztest(successes, trials)
    print(f"{step:8} | Test: {test_success}/{total_test} | Control: {control_success}/{total_control} | p-value = {pval:.4f} ", end="")

    if pval < 0.05:
        print("=> Significant difference")
    else:
        print("=> No significant difference")

Total Control visits: 30741
Total Test visits: 33017

Step-wise significance test (two-proportion z-test):

start    | Test: 33017/33017 | Control: 30741/30741 | p-value = nan => No significant difference
step_1   | Test: 28148/33017 | Control: 23394/30741 | p-value = 0.0000 => Significant difference
step_2   | Test: 24378/33017 | Control: 19997/30741 | p-value = 0.0000 => Significant difference
step_3   | Test: 22069/33017 | Control: 18165/30741 | p-value = 0.0000 => Significant difference
confirm  | Test: 21608/33017 | Control: 15912/30741 | p-value = 0.0000 => Significant difference


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std


THE TEST GROUP HAS A SIGNIFICANTLY HIGHGER CONVERSION RATIOS FOR EACH STEP

2) COMPLETION RATE WITH A COST EFFECTIVENESS THRESHOLD

In [10]:
# Completion rates at 'confirm' step from your pivot table
completion_control = pivot.loc['confirm', 'Control'] / total_control * 100
completion_test = pivot.loc['confirm', 'Test'] / total_test * 100

# Calculate the lift
completion_lift = completion_test - completion_control

# Check threshold
threshold = 5.0  # minimum % increase required

print(f"Control completion rate: {completion_control:.2f}%")
print(f"Test completion rate: {completion_test:.2f}%")
print(f"Lift in completion rate: {completion_lift:.2f}%")

if completion_lift >= threshold:
    print(f"✅ The lift meets or exceeds the cost-effectiveness threshold of {threshold}%.")
else:
    print(f"⚠️ The lift is below the cost-effectiveness threshold of {threshold}%.")

Control completion rate: 51.76%
Test completion rate: 65.45%
Lift in completion rate: 13.68%
✅ The lift meets or exceeds the cost-effectiveness threshold of 5.0%.


3) OTHER HYPOTHESIS: GENDER DIFERENCES IN COMPLETION RATES

In [14]:
# Assume your funnel step data with client_id is in `df`
# And demographics_df contains client_id and gender

# Merge funnel data with demographics to get gender per visit
df_merged = df.merge(demographics_df[['client_id', 'gender']], on='client_id')

# Remove duplicate steps per visit to avoid counting loops multiple times
df_unique_steps = df_merged.drop_duplicates(subset=['visit_id', 'process_step'])

# Define funnel steps order
step_order = ["start", "step_1", "step_2", "step_3", "confirm"]

results = []

# Calculate counts and conversion rates by gender and step
for step in step_order:
    step_data = df_unique_steps[df_unique_steps['process_step'] == step]
    
    counts = step_data.groupby('gender')['visit_id'].nunique()
    total_starts = df_unique_steps[df_unique_steps['process_step'] == 'start'].groupby('gender')['visit_id'].nunique()
    
    for gender in ['M', 'F']:
        step_count = counts.get(gender, 0)
        start_count = total_starts.get(gender, 0)
        conversion_rate = step_count / start_count if start_count > 0 else 0
        
        results.append({
            'step': step,
            'gender': gender,
            'step_count': step_count,
            'start_count': start_count,
            'conversion_rate': conversion_rate
        })

results_df = pd.DataFrame(results)

# Print conversion rates by gender
print("Conversion rates by step and gender:\n")
for step in step_order:
    data_step = results_df[results_df['step'] == step]
    male_rate = data_step[data_step['gender'] == 'M']['conversion_rate'].values[0] * 100
    female_rate = data_step[data_step['gender'] == 'F']['conversion_rate'].values[0] * 100
    print(f"{step}: Male = {male_rate:.2f}%, Female = {female_rate:.2f}%")

print("\nStatistical significance tests (two-proportion z-test):\n")

# Run two-proportion z-test for each step
for step in step_order:
    data_step = results_df[results_df['step'] == step]
    successes = data_step['step_count'].values
    trials = data_step['start_count'].values
    
    stat, pval = proportions_ztest(successes, trials)
    
    male_count, female_count = successes
    male_total, female_total = trials
    
    print(f"{step}: Male {male_count}/{male_total} vs Female {female_count}/{female_total} | p-value = {pval:.4f} -> ", end='')
    
    if pval < 0.05:
        print("Significant difference")
    else:
        print("No significant difference")

Conversion rates by step and gender:

start: Male = 100.00%, Female = 100.00%
step_1: Male = 80.62%, Female = 80.30%
step_2: Male = 68.85%, Female = 67.99%
step_3: Male = 62.38%, Female = 62.25%
confirm: Male = 60.14%, Female = 57.22%

Statistical significance tests (two-proportion z-test):

start: Male 30440/30440 vs Female 29024/29024 | p-value = nan -> No significant difference
step_1: Male 24542/30440 vs Female 23306/29024 | p-value = 0.3175 -> No significant difference
step_2: Male 20959/30440 vs Female 19732/29024 | p-value = 0.0228 -> Significant difference
step_3: Male 18989/30440 vs Female 18068/29024 | p-value = 0.7440 -> No significant difference
confirm: Male 18306/30440 vs Female 16608/29024 | p-value = 0.0000 -> Significant difference


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/weightstats.py:792: RuntimeWarning: invalid value encountered in scalar divide
  zstat = value / std
